In [1]:
import os
import sys
import json
import stanza
from tqdm import tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work/ser'

In [25]:
import xml.etree.ElementTree as ET
import difflib
import re
from collections import Counter

In [3]:
import ser_annotate.ssqa_util as Util
from ser_annotate.std import *
from ser_annotate.util import *

In [27]:
_log = Logger(__name__)

In [8]:
def read_dats(f):
    myPs, myQs = [], []
    myQ = []
    for myLine in (x.rstrip() for x in f):
        # _log.debug('>>{}<<'.format(myLine))
        if not myLine: continue  # pass the empty line
        if myLine[0] == 'P':  # read lesson text
            if myQs:
                yield myPs, myQs
                myPs, myQs = [], []
            myKey, myTxt = myLine.split('\t')
            myPs.append([myKey, myTxt])
        elif myLine[0] == '0':
            assert not myQ
            myQ = myLine.split('\t')
            assert len(myQ) == 2
        elif myLine[0] == 'b':
            assert len(myQ) == 2
            myKey, *myToks = myLine.split('\t')
            if len(myToks) == 1 and myToks[0] == 'NONE':
                myQs.append(myQ)
                myQ = []
            else:
                for iTok in myToks:
                    myKey, myTxt = iTok.split()
                    assert myKey.startswith('Pub')
                    myQ.append([myKey, myTxt])
                myQs.append(myQ)
                myQ = []
    yield myPs, myQs

In [13]:
def get_parag_idx(myP):
    outIdx = []
    myKey, myTxt = myP
    mySegTxt = string_split(myTxt)
    outIdx.extend(list(range(0, len(mySegTxt))))
    return outIdx

In [14]:
def read_old_benchmark(f):
    totQsNum = 0
    myLessons = []
    for i, (myPs, myQs) in enumerate(read_dats(f), 1):
        myFid2Bens = {}
        myFid2Parg = []
        myBens = []
        # lprint(len(myQs))  # the number of questions in each lesson
        myFid = None
        for iP in myPs:
            myKey, myTxt = iP
            myIdx = get_parag_idx(iP)
            myFid2Parg.append([myKey, myIdx])
        for iQ in myQs:
            myKey, myTxt, *myEvidParts = iQ
            for iEvidPart in myEvidParts:
                a, b, c, d = iEvidPart[0].split('-')
                if myFid is None:
                    myFid = '{}-{}-{}'.format(a, b, c)
                assert myFid == '{}-{}-{}'.format(a, b, c)
                iEvidPart.append(get_idx(myPs, iEvidPart))
            myBens.append([myKey, myTxt, myEvidParts])
        myLessons.append([myFid2Parg, myBens])
        totQsNum += len(myQs)
    lprint(len(myLessons))
    lprint(totQsNum)
    return myLessons

In [21]:
def get_idx(iPs, iBen):
    outIdx = []
    benKey, benTxt = iBen
    benKeyNum = int(re.search('Pub.-G\d[ab]-\d{4}-(\d{3})', benKey).group(1))
    # if re.match('.*\|{3,}.*',benTxt):
    # lprint("match|||")
    myBenSegs = string_split(re.sub('\|{3,}', '', benTxt))
    iPs[benKeyNum - 1]
    myParSegs = string_split(iPs[benKeyNum - 1][1])
    mySeqMat = difflib.SequenceMatcher(None, myBenSegs, myParSegs)
    for tag, i1, i2, j1, j2 in mySeqMat.get_opcodes():
        _log.debug('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
            tag, i1, i2, j1, j2, myBenSegs[i1:i2], myParSegs[j1:j2]))
    for tag, i1, i2, j1, j2 in mySeqMat.get_opcodes():
        if tag == 'equal':
            outIdx.extend(list(range(j1, j2)))
        else:
            if tag != 'insert':
                raise ValueError('{}\n{}\n'.format(myBenSegs, myParSegs))
    # if re.match('.*\|{3,}.*', benTxt):
    # lprint(outIdx)
    return outIdx

In [22]:
_PUNCT_SET = frozenset(["，", ",", "、", "；", "。", "：", "！", "？", "?"])
def string_split(str):
    myOut = []
    mySeg = ''
    for iChar in str:
        mySeg += iChar
        if iChar in _PUNCT_SET:
            myOut.append(mySeg)
            mySeg = ''
    if mySeg:
        myOut.append(mySeg)
    return myOut

In [29]:
list_fps('./data/SSQA/SSQA_benchmark_v3.2_0312/', '3.2')

['./data/SSQA/SSQA_benchmark_v3.2_0312/dev_207_HumanAnnotation_wLessonText_V.3.2',
 './data/SSQA/SSQA_benchmark_v3.2_0312/dev_207_HumanAnnotation_woLessonText_V.3.2',
 './data/SSQA/SSQA_benchmark_v3.2_0312/test_207_HumanAnnotation_wLessonText_V.3.2',
 './data/SSQA/SSQA_benchmark_v3.2_0312/test_207_HumanAnnotation_woLessonText_V.3.2',
 './data/SSQA/SSQA_benchmark_v3.2_0312/train_202_HumanAnnotation_wLessonText_V.3.2',
 './data/SSQA/SSQA_benchmark_v3.2_0312/train_202_HumanAnnotation_woLessonText_V.3.2']

In [31]:
# with open('./data/SSQA/SSQA_benchmark_v3.2_0312/dev_207_HumanAnnotation_wLessonText_V.3.2') as f:
#     read_old_benchmark(f)

In [20]:
read_old_benchmark()

TypeError: read_old_benchmark() missing 1 required positional argument: 'f'

In [4]:
def _leaf2text(p):
    assert len(p) == 0
    myText = p.text.strip(' \t\r\n')
    insist(len(myText) > 0)
    insist(myText == p.text)
    insist(myText.find('\n') < 0)
    insist(myText.find('\r') < 0)
    insist(myText.find('\t') < 0)
    return myText

In [5]:
def _body2dat(p, dat, xpath=''):
    # for c in p: print(c.tag, c.attrib)
    myXpath = xpath
    if myXpath: myXpath += '/'
    myXpath += p.tag

    if len(p) == 0:
        # terminal
        dat.append((myXpath, _leaf2text(p)))
    else:
        # nonterminal
        for c in p: _body2dat(c, dat, myXpath)

In [48]:
def _qst2dat(qsts):
    type_dict = {}
    for q_i, q in enumerate(qsts):
        assert q.tag == 'QA'
        qtype = q.attrib['Type']
        q_idx = q.attrib['idx']
        
        if qtype not in type_dict:
            type_dict[qtype] = 1
        else:
            type_dict[qtype] += 1
    return type_dict

In [53]:
def lesson_xml2txt(fhs, type_counter):
    outFh = 'Lesson.txt'

    myFid, myGrad = re.search('/(Pub.-G(\d)[ab]-\d{4})\.', fhs).groups()
    # lprint(myFid)
    tree = ET.parse(fhs)
    p = tree.getroot()
    for i in ['Machine-Reading-Corpus-File', 'Content', 'Unit']:
        assert p.tag == i
        if i != 'Unit':
            assert len(p) == 1
            p = p[0]
    assert len(p) == 2  # Body, QAset

    # read the lessons
    myBodyDat = []
    assert p[0].tag == 'Body'
    _body2dat(p[0], myBodyDat)
#     lprint(myBodyDat)
    for i, (iTag, iTxt) in enumerate(myBodyDat):
        Util.check_unescaped_text(iTxt)
        assert iTxt.find('&') < 0
        myTxt = Util.xml_escape(iTxt)
        Util.check_escaped_text(myTxt)
        if myTxt != iTxt:
            lprint('{}\n\t{}\n\t{}\n'.format(myFid, iTxt, myTxt))
#         with open(outFh, 'a', encoding='utf-8') as f:
#             f.write(myTxt + '\n')
        
    # read the questions
    myQstDat = []
    assert p[1].tag == 'QAset'
    q_num = int(p[1].attrib['Num_Ques'])
    
    assert q_num == len(p[1]), 'question num incorrect'
    type_dict = _qst2dat(p[1])
    type_counter.update(type_dict)

In [71]:
ssqa_fp = './data/SSQA/Elementary_Social_Studies_v2.9/'

In [72]:
test_file = './data/SSQA/Elementary_Social_Studies_v2.9/Test/PubB-G3a-0301.xml'

In [73]:
type_counter = Counter()
lesson_xml2txt(test_file, type_counter)

In [74]:
set_list = ['Train', 'Develop', 'Test']

In [75]:
q_num = 0

for i_set in set_list:
    type_counter = Counter()
    for fp in tqdm(os.listdir(ssqa_fp+i_set)):
        fp_c = ssqa_fp+i_set + '/' + fp
        try:
            p = ET.parse(fp_c).getroot()
            assert p[0][0][1].tag == 'QAset'
            q_num += len(p[0][0][1])
            lesson_xml2txt(fp_c, type_counter)
        except Exception as e:
            print(e)
            print(fp_c)
    print(i_set)
    print(type_counter)

 37%|███▋      | 77/210 [00:00<00:00, 765.31it/s]<ipython-input-53-3c246c7e2016>(26): PubC-G3b-0602
	對話框:"各組想想看，有什麼解決的方法？">>"我們可以寫信給區長，請他幫忙。">>"製作海報張貼在社區。">>"請學校發起淨灘活動。"
	對話框:"各組想想看，有什麼解決的方法？"&gt;&gt;"我們可以寫信給區長，請他幫忙。"&gt;&gt;"製作海報張貼在社區。"&gt;&gt;"請學校發起淨灘活動。"

100%|██████████| 27/27 [00:00<00:00, 599.12it/s]

Train
Counter({'true-false': 11131, 'multiple-choice': 4022, 'essay': 999, 'multiple-select': 191})
Develop
Counter({'true-false': 1508, 'multiple-choice': 789, 'essay': 208, 'multiple-select': 81})
Test
Counter({'true-false': 1464, 'multiple-choice': 788, 'essay': 254, 'multiple-select': 68})


In [245]:
q_num

21503

In [29]:
p = ET.parse(ssqa_fp+'Develop'+'/PubB-G3a-0303.xml').getroot()

In [31]:
len(p[0][0][1])

114